翻訳器の作成．
英⇒日はwebから引用したものを利用.
日⇒英は100本ノック98で作ったものを利用する.

まずは必要なデータやライブラリをそろえます．

参考 
https://qiita.com/nymwa/items/2f39a34982aa9d71f10d  
https://gist.github.com/nymwa/d584be8ed36b9513a31e31f5b8f50112  
https://qiita.com/O-Kazu-O/items/5c0781913a1d033bea39

## ここから英→日の訓練　訓練は別サーバで行った

In [ ]:
!git clone https://github.com/nymwa/light_enja2.git

In [ ]:
!mv light_enja2/* .

In [ ]:
!pip install -r requirements.txt

データに対して前処理を行います

ダウンロードした時点でデータセットは単語分割やトークン化がすでに施されています．
マージ数8000のBPEでサブワードに分割します．

In [ ]:
! bash preproc.sh

学習を行います．

少ないデータでかつかなり軽量なモデルでの学習ですが，10分ぐらいかかります．

In [ ]:
!fairseq-train \
    data-bin \
    --save-interval 10 \
    --max-epoch 10 \
    --update-freq 1 \
    --max-tokens 4000 \
    --arch transformer \
    --encoder-normalize-before \
    --decoder-normalize-before \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 1024 \
    --encoder-attention-heads 4 \
    --encoder-layers 4 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 1024 \
    --decoder-attention-heads 4 \
    --decoder-layers 4 \
    --share-all-embeddings \
    --dropout 0.3 \
    --optimizer adam \
    --adam-betas '(0.9, 0.999)' \
    --lr 0.002 \
    --lr-scheduler inverse_sqrt \
    --warmup-updates 2000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 1.0 \
    --weight-decay 0.01 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.6

テストデータで翻訳を行い，その性能を評価します．

評価尺度BLEUで30ぐらい出ると思います．

In [ ]:
! fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint10.pt \
    --beam 5 \
    --lenpen 0.6 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu corpus/test.ja

翻訳結果と正解データの冒頭10文です．

In [ ]:
! head output.txt corpus/test.ja

## ここまで訓練
### ここからlocalで実行するために必要なものをインストールする

In [ ]:
pip install fairseq==0.10.0

In [ ]:
pip install -U sacremoses

In [ ]:
pip install sentencepiece

In [ ]:
pip install --upgrade --user numpy

# 実践(localで)

In [ ]:
import re
import unicodedata
from sacremoses import MosesTokenizer
import sentencepiece as spm
from fairseq.models.transformer import TransformerModel

In [ ]:
mt = MosesTokenizer(lang = 'en')
sp = spm.SentencePieceProcessor(model_file='bpe.model')
ejmodel = TransformerModel.from_pretrained('./checkpoints/', checkpoint_file='checkpoint10.pt', data_name_or_path='data-bin')

def preproc_en(x):
  x = unicodedata.normalize('NFKC', x)
  x = re.sub(mt.AGGRESSIVE_HYPHEN_SPLIT[0], r'\1 - ', x)
  x = mt.tokenize(x, escape = False)
  x = ' '.join(x)
  x = x.lower()
  x = ' '.join(sp.encode(x, out_type = 'str'))
  return x

def ejtranslate(x):
  x = preproc_en(x)
  x = ejmodel.translate(x, beam = 5, lenpen = 0.6)
  x = ''.join(x.split()).replace('▁', '').strip()
  return x

In [ ]:
while True:
  x = input('英文を入力 > ')
  if not x:
    break
  x = ejtranslate(x)
  print('翻訳結果 > ' + x)

In [ ]:
from fairseq.models.transformer import TransformerModel

In [ ]:
jemodel = TransformerModel.from_pretrained('./save98_2/', checkpoint_file='checkpoint10.pt', data_name_or_path='data98_2')

In [ ]:
import MeCab
import sys
import re 

In [ ]:
def jetranslate(x):
    wakati = MeCab.Tagger('-Owakati')
    wakati = ' '.join(wakati.parse(x).split())
    x = jemodel.translate(wakati, beam = 5, lenpen = 0.6) 
    x = re.sub('@@ ?$', '', x)
    x = re.sub('@@ ', '', x)
    return x

In [ ]:
text = "私は、道元です。"
print(jetranslate(text))

In [ ]:
from flask import Flask, render_template, request

# WEBサーバ　翻訳機
jupyter notebookでプログラム実行後

`ngrok http 5000`

をpowershellなどで実行する

In [ ]:
app = Flask(__name__)

@app.route("/", methods=["GET"])
def get():
    return render_template("index.html", \
        title = "日⇔英翻訳機", \
        message = "Please enter the text you want to translate here.(翻訳したい文を入れてください)",\
        ejmessage = "",\
        jemessage = "")

@app.route("/", methods=["POST"])
def post():
    ejtext = request.form["ejname"]
    #if re.compile(\s*).search(ejtext):
    if ejtext =="":
     ejoutput =""
    else:
     ejoutput = ejtranslate(ejtext)
    jetext = request.form["jename"]
    if jetext =="":
     jeoutput =""
    else:    
     jeoutput = jetranslate(jetext)
    return render_template("index.html",\
        title = "日⇔英翻訳機(出力結果)",\
        message = "翻訳完了",\
        ejmessage = "英→日結果:　　"+ejtext+"　　→　　"+ejoutput ,\
        jemessage = "日→英結果:　　"+jetext+"　　→　　"+jeoutput)
        

app.run()

## test用

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
  return 'Hello World!'

if __name__ == '__main__':
  app.run()